In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
import random

# Definir uma semente aleatória para reprodutibilidade.
np.random.seed(42)
random.seed(42)

In [17]:
# Gerar as variáveis simuladas com correlação.
idade = np.random.randint(18, 71, 10000)

# Gerar variáveis correlacionadas usando a função multivariada normal.
mean_values = [5000, 2000, 0.5, 5]  # Médias das variáveis.
correlation_matrix = np.array([
    [1, 0.3, 0.2, -0.1],
    [0.3, 1, -0.1, 0.2],
    [0.2, -0.1, 1, 0.4],
    [-0.1, 0.2, 0.4, 1]
])  # Matriz de correlação.

# Gerar dados simulados
simulated_data = np.random.multivariate_normal(mean_values, correlation_matrix, 10000)

renda = simulated_data[:, 0]
divida = simulated_data[:, 1]
utilizacao_credito = np.clip(simulated_data[:, 2], 0, 1)  # Limita a utilização de crédito entre 0 e 1.
consultas_recentes = np.maximum(simulated_data[:, 3], 0)  # Garante que o número de consultas recentes seja não negativo.

# Gerar função linear das variáveis explicativas.
preditor_linear = -7 - 0.01 * idade - 0.0002 * renda + 0.003 * divida - 3 * utilizacao_credito + 0.5 * consultas_recentes

# Calcular probabilidade de default (PD) usando a função de link logit.
prob_default = 1 / (1 + np.exp(-preditor_linear))

# Gerar inadimplência como variável Bernoulli com base na probabilidade de default.
inadimplencia = np.random.binomial(1, prob_default, 10000)

# Criar dataframe.
dados = pd.DataFrame({
    'idade': idade,
    'renda': renda,
    'divida': divida,
    'utilizacao_credito': utilizacao_credito,
    'consultas_recentes': consultas_recentes,
    'inadimplencia': inadimplencia
})

dados.head()

,idade,renda,divida,utilizacao_credito,consultas_recentes,inadimplencia
0,18,5002.022300,1999.201036,0.785374,4.202680,0
1,57,4999.201718,2000.017719,0.000000,4.422066,0
2,55,5000.198873,1999.426202,1.000000,7.207097,0
3,51,5001.006507,1999.608526,1.000000,5.378260,1
4,70,5002.155973,2001.666174,0.379364,5.508144,0


In [20]:
# Categorizar a idade.
kbin = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
dados['idade_cat'] = kbin.fit_transform(dados[['idade']])
dados.head()

,idade,renda,divida,utilizacao_credito,consultas_recentes,inadimplencia,idade_cat
0,18,5002.022300,1999.201036,0.785374,4.202680,0,0.0
1,57,4999.201718,2000.017719,0.000000,4.422066,0,3.0
2,55,5000.198873,1999.426202,1.000000,7.207097,0,3.0
3,51,5001.006507,1999.608526,1.000000,5.378260,1,3.0
4,70,5002.155973,2001.666174,0.379364,5.508144,0,4.0


In [21]:
def descritiva2(var1, var2, df):
    cross_tab = pd.crosstab(df[var1], df[var2], normalize='index')
    print(cross_tab)

descritiva2('idade_cat', 'inadimplencia', dados)

inadimplencia         0         1
idade_cat                        
0.0            0.719936  0.280064
1.0            0.728117  0.271883
2.0            0.742912  0.257088
3.0            0.775934  0.224066
4.0            0.776867  0.223133


In [22]:
dados.head()

,idade,renda,divida,utilizacao_credito,consultas_recentes,inadimplencia,idade_cat
0,18,5002.022300,1999.201036,0.785374,4.202680,0,0.0
1,57,4999.201718,2000.017719,0.000000,4.422066,0,3.0
2,55,5000.198873,1999.426202,1.000000,7.207097,0,3.0
3,51,5001.006507,1999.608526,1.000000,5.378260,1,3.0
4,70,5002.155973,2001.666174,0.379364,5.508144,0,4.0
